In [1]:
import pandas as pd
import altair as alt

# Data Transformation for OBR Uncertainty Charts

In [40]:
import pandas as pd
import altair as alt

gdp_for = pd.read_excel('Chapter_2_charts_and_tables_November_2025.xlsx', sheet_name='C2.10', skiprows=24)


# Rename columns: we have to do this as the first row contains part of the column names (they're split across two rows)
gdp_for.columns = [f"{col}_{val}" for col, val in zip(gdp_for.columns, gdp_for.iloc[0])]
gdp_for = gdp_for.drop(0).reset_index(drop=True)

gdp_for.drop(columns=['Unnamed: 0_nan'], inplace=True)

gdp_for = gdp_for.rename(columns={'Unnamed: 1_nan': 'Year', 
                        'March 2025 forecast_nan': 'March 2025 Forecast', 
                        'November 2025 forecast_nan': 'November 2025 Forecast',
                        'Percentile_10.0': 'Percentile_10',
                        'Unnamed: 5_20.0': 'Percentile_20',
                        'Unnamed: 6_30.0': 'Percentile_30',
                        'Unnamed: 7_40.0': 'Percentile_40',
                        'Unnamed: 8_50.0': 'Percentile_50',
                        'Unnamed: 9_60.0': 'Percentile_60',
                        'Unnamed: 10_70.0': 'Percentile_70',
                        'Unnamed: 11_80.0': 'Percentile_80',
                        'Unnamed: 12_90.0': 'Percentile_90'},)



# Cleaner way using cumsum
percentile_cols = ['Percentile_10', 'Percentile_20', 'Percentile_30', 'Percentile_40', 
                   'Percentile_50', 'Percentile_60', 'Percentile_70', 'Percentile_80', 'Percentile_90']

gdp_for[percentile_cols] = gdp_for[percentile_cols].cumsum(axis=1)

# Fill NaNs in the 2024 row with the November 2025 Forecast	 value
gdp_for.loc[gdp_for['Year'] == 2024, percentile_cols] = gdp_for.loc[gdp_for['Year'] == 2024, 'November 2025 Forecast'].values[0]
gdp_for['Year'] = pd.to_datetime(gdp_for['Year'], format='%Y')

# Reshape uncertainty data
uncertainty_df = pd.concat([
    gdp_for[['Year', 'Percentile_10', 'Percentile_90']].assign(band='10-90').rename(
        columns={'Percentile_10': 'lower', 'Percentile_90': 'upper'}),
    gdp_for[['Year', 'Percentile_20', 'Percentile_80']].assign(band='20-80').rename(
        columns={'Percentile_20': 'lower', 'Percentile_80': 'upper'}),
    gdp_for[['Year', 'Percentile_30', 'Percentile_70']].assign(band='30-70').rename(
        columns={'Percentile_30': 'lower', 'Percentile_70': 'upper'})
], ignore_index=True)

# Reshape forecast data
forecast_df = pd.concat([
    gdp_for[['Year', 'March 2025 Forecast']].assign(forecast='March 2025').rename(
        columns={'March 2025 Forecast': 'value'}),
    gdp_for[['Year', 'November 2025 Forecast']].assign(forecast='November 2025').rename(
        columns={'November 2025 Forecast': 'value'})
], ignore_index=True)



In [48]:
forecast_df

,Year,value,forecast
0,2022-01-01,4.839085,March 2025
1,2023-01-01,0.382437,March 2025
2,2024-01-01,0.935022,March 2025
3,2025-01-01,0.971593,March 2025
4,2026-01-01,1.861722,March 2025
5,2027-01-01,1.790356,March 2025
6,2028-01-01,1.722626,March 2025
7,2029-01-01,1.800816,March 2025
8,2030-01-01,NaN,March 2025
9,2022-01-01,5.149704,November 2025


In [49]:
uncertainty_df.dropna().to_csv('obr_growth_uncertainty_ranges.csv', index=False)
forecast_df.dropna().to_csv('obr_growth_forecasts.csv', index=False)

# The Charts

In [50]:

vgl_spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "UK Growth Forecasts",
    "width": 600,
    "height": 400,
    "layer": [
        {
            "data": {"url": "https://raw.githubusercontent.com/jhellingsdata/RADataHub/refs/heads/main/misc/Forecast_Uncertainty/obr_growth_uncertainty_ranges.csv"},
            "mark": {"type": "area"},
            "encoding": {
                "x": {"field": "Year", "type": "temporal", "title": "Year"},
                "y": {"field": "lower", "type": "quantitative", "title": "GDP Forecast"},
                "y2": {"field": "upper"},
                "color": {
                    "field": "band",
                    "type": "nominal",
                    "scale": {
                        "domain": ["10-90", "20-80", "30-70"],
                        "range": ["rgba(54, 183, 181, 0.1)", "rgba(54, 183, 181, 0.2)", "rgba(54, 183, 181, 0.6)"]
                    },
                    "legend": None
                },
                "order": {
                    "field": "band",
                    "type": "ordinal",
                    "sort": "descending"
                }
            }
        },
                        {
            "data": {"url": "https://raw.githubusercontent.com/jhellingsdata/RADataHub/refs/heads/main/misc/Forecast_Uncertainty/obr_growth_forecasts.csv"},
            "mark": {"type": "line", "strokeWidth": 2},
            "encoding": {
                "x": {"field": "Year", "type": "temporal"},
                "y": {"field": "value", "type": "quantitative"},
                "color": {
                    "field": "forecast",
                    "type": "nominal",
                    "scale": {
                        "range": ["#f4c245", "#179fdb"]
                    },
                },
            }
        },
    ],
    "title": {
        "text": "GDP Forecast with Percentile Ranges",
        "fontSize": 16,
        "anchor": "start",
        "color": "#000000"
    },
    "resolve": {
        "scale": {
            "color": "independent"
        }
    }
}

gdp_forecast_chart = alt.Chart.from_dict(vgl_spec)
gdp_forecast_chart

alt.LayerChart(...)